In [66]:
class Base:
    def __init__(self):
        pass
    
    @classmethod
    def inherited_from(cls, parent_cls):
        if parent_cls in cls.__mro__:
            return True
        else:
            raise cls.NotInheritedError()
            
    class NotInheritedError(Exception):
        pass

In [104]:
class Strategy(Base):
    def __init__(self, data=FinancialData_KR):
        pass
    
    def bm(self):
        pass

In [69]:
class Data(Base):
    def __init__(self):
        pass
    
    def price(self):
        pass

In [103]:
class FinancialData_KR(Data):
    def __init__(self):
        pass

In [70]:
class ProfitTake(Base):
    def __init__(self):
        pass

In [71]:
class Losscut(Base):
    def __init__(self):
        pass

In [72]:
class Review(Base):
    def __init__(self):
        pass

In [94]:
class Backtester:
    def __init__(self, strategy=Strategy, profittake=ProfitTake, losscut=Losscut, review=Review):
        self.strategy = self._modulize(strategy, Strategy)
        self.price = self._modulize(price, Price)
        self.data = self._modulize(data, Data)
        self.profittake = self._modulize(profittake, ProfitTake)
        self.losscut = self._modulize(losscut, Losscut)
        self.review = self._modulize(review, Review)
        
    def _modulize(self, module, parent_cls):
        if module.inherited_from(parent):
            return module
        
    def run(self):
        pass

In [88]:
bt = Backtester()

In [97]:
Backtester.__mro__

(__main__.Backtester, object)